In [ ]:
import pandas as pd
import geopandas as gpd
import glob
import re
from shapely.geometry import Point
import os
from tqdm import tqdm
import shutil

In [ ]:
# Configs

huc_code = '14'
huc_feature = 'huc2'
feature_name = 'comid'


In [ ]:
# Functions

def vcat(pattern, subset_dedup=None):
    """Read all CSVs matching pattern and vertically concatenate."""
    files = glob.glob(pattern)
    if not files:
        raise FileNotFoundError(f"No files matched: {pattern}")
    dfs = [pd.read_csv(f) for f in files]
    out = pd.concat(dfs, ignore_index=True)
    if subset_dedup:
        out = out.drop_duplicates(subset=subset_dedup)
    return out

def closest_segment_POD(point, gdf):
    global fallback3_count, fallback3_ids
    point_geom = point.geometry
    point_gnis_id = point['SOURCE_GNIS_ID']
    point_gnis_id_raw = str(point['Source GNIS ID Raw']).strip().lower()
    point_source = str(point['WATER_SOURCE']).strip().lower()
    pod_id = point.get('WDID', None)

    def normalize_name(name):
        if not isinstance(name, str):
            return ''
        name = name.lower().strip()

        # Remove directional prefixes
        directional_prefixes = ['east ', 'west ', 'north ', 'south ', 'upper ', 'lower ']
        for p in directional_prefixes:
            if name.startswith(p):
                name = name[len(p):]

        # Remove hydrologic suffixes
        suffixes = [' river', ' creek', ' stream', ' wash', ' arroyo', ' fork', ' branch']
        for s in suffixes:
            if name.endswith(s):
                name = name[:-len(s)]

        return name.strip()

    # Case 1: Use GNIS ID directly
    if pd.notna(point_gnis_id) and point_gnis_id_raw != 'tbd':
        matching_segments = gdf[gdf['gnis_id'] == point_gnis_id].copy()
        if matching_segments.empty:
            matching_segments = gdf.copy()
        matching_segments['distance'] = matching_segments.distance(point_geom)
        closest_idx = matching_segments['distance'].idxmin()
        return matching_segments.loc[closest_idx]

    # Case 2: Use normalized Water Source Name
    elif point_source != 'tbd':

        normalized_source = normalize_name(point_source)
        candidate_segments = gdf[gdf['gnis_name'].notna()].copy()

        def name_matches(gnis):
            return normalize_name(gnis) == normalized_source

        candidate_segments['name_match'] = candidate_segments['gnis_name'].apply(name_matches)
        matched_segments = candidate_segments[candidate_segments['name_match']]

        if matched_segments.empty:
            matched_segments = gdf[gdf['streamorde'].fillna(0) >= 3].copy()
            fallback3_count += 1
            if pod_id is not None:
                fallback3_ids.append(pod_id)

        matched_segments = matched_segments.assign(
            distance=matched_segments.geometry.distance(point_geom)
        )
        closest_idx = matched_segments['distance'].idxmin()
        return matched_segments.loc[closest_idx]

    # Case 3: Fallback to streamorde >= 3
    else:
        fallback_segments = gdf[gdf['streamorde'].fillna(0) >= 3].copy()
        fallback3_count += 1
        if pod_id is not None:
            fallback3_ids.append(pod_id)

        if fallback_segments.empty:
            print("⚠️ Fallback segments are empty — using full GDF")
            fallback_segments = gdf.copy()
        fallback_segments['distance'] = fallback_segments.distance(point_geom)
        closest_idx = fallback_segments['distance'].idxmin()
        return fallback_segments.loc[closest_idx]


def closest_segment_func(point, gdf):
    lake_segments = gdf[gdf['wbareatype'] == 'LakePond'].copy()
    lake_segments.loc[:, 'distance'] = lake_segments.distance(point)
    closest_idx = lake_segments['distance'].idxmin()
    return lake_segments.loc[closest_idx]

res_fallback_counter= 0
def closest_segment_func2(point, gdf):
    global res_fallback_counter
    """
    Link a reservoir to the nearest LakePond segment, preferring name matches
    on the 'RIVER' field when available. Robust to missing/None river names.
    """
    # --- helpers ---
    def safe_str(x):
        return "" if pd.isna(x) or x is None else str(x)

    def normalize_name(name):
        name = safe_str(name).lower().strip()
        if not name:
            return name
        # Remove common prefixes/suffixes and descriptors
        prefixes = [
            'east ', 'west ', 'north ', 'south ', 'upper ', 'lower ',
            'middle fork of ', 'middle ', 'offstream ', 'off-stream ',
        ]
        for p in prefixes:
            if name.startswith(p):
                name = name[len(p):]
        suffixes = [' river', ' creek', ' stream', ' wash', ' arroyo', ' fork', ' branch', ' canal']
        for s in suffixes:
            if name.endswith(s):
                name = name[:-len(s)]
        return name.strip()

    river_name_norm = normalize_name(point.get('RIVER', ''))

    # Candidate segments: lakes/ponds with a name
    candidate_segments = gdf[
        (gdf['wbareatype'] == 'LakePond') & (gdf['gnis_name'].notna())
    ].copy()

    # If there is a usable river name, try a strict normalized match first
    if river_name_norm:
        candidate_segments = candidate_segments.assign(
            _name_norm=candidate_segments['gnis_name'].apply(normalize_name)
        )
        matched = candidate_segments[candidate_segments['_name_norm'] == river_name_norm]

        # If nothing strict-matched, try a softer contains match (handles things like "Offstream Little Sandy Creek")
        if matched.empty:
            matched = candidate_segments[candidate_segments['_name_norm'].str.contains(river_name_norm, na=False)]

        # If there are any name-based matches, choose the nearest of those
        if not matched.empty:
            matched = matched.assign(distance=matched.geometry.distance(point.geometry))
            return matched.loc[matched['distance'].idxmin()]

    # Fallbacks (no/poor name or no matches):
    # 1) Restrict spatially around the reservoir to avoid whole-basin scan
    buf = gpd.GeoSeries([point.geometry], crs=gdf.crs).buffer(10_000).iloc[0]
    spatial = gdf[(gdf['wbareatype'] == 'LakePond') & gdf.intersects(buf)].copy()
    if not spatial.empty:
        spatial = spatial.assign(distance=spatial.geometry.distance(point.geometry))
        res_fallback_counter += 1
        return spatial.loc[spatial['distance'].idxmin()]

    # 2) Final fallback: nearest LakePond anywhere
    lake_segments = gdf[gdf['wbareatype'] == 'LakePond'].copy()
    if lake_segments.empty:
        return None
    lake_segments = lake_segments.assign(distance=lake_segments.geometry.distance(point.geometry))
    return lake_segments.loc[lake_segments['distance'].idxmin()]

In [ ]:
# Directories

sites_df  = vcat("data/wade data/*/sites.csv", subset_dedup=["SiteUUID"])
alloc_df  = vcat("data/wade data/*/waterallocations.csv", subset_dedup= None)
source_df = vcat("data/wade data/*/watersources.csv", subset_dedup=["WaterSourceUUID"])


diversions_df = pd.read_csv('data/Diversions/diversion_points_lopez.csv')

agg_csv= 'data/*' # If you have aggregated diversions with correct format (see example) and place them in data folder 


wbd_layer = gpd.read_file('data/wbd/WBDHU2.shp')

# Load gage locations
gage_shp = gpd.read_file('data/GageLoc/GageLoc.shp') # This is optional, it is nod added to the extended hydrofabric
reference_fabric_path = 'data/reference fabric gpkg/reference_14.gpkg'
flow_layer = gpd.read_file(reference_fabric_path, layer='reference_flowline')
gage_data = gpd.read_file(reference_fabric_path, layer='event')
# Load reservoir datasets
resops_df = pd.read_csv('data/reservoirs/updated_reservoir_attributes.csv')
grand_df  = pd.read_csv('data/reservoirs/GRAND.csv')

In [ ]:
# WaDE data Harmonizing

alloc_norm = alloc_df.copy()

# split lists, explode, and clean whitespace/empties
alloc_norm['SiteUUID'] = (
    alloc_norm['SiteUUID']
    .astype(str)
    .str.split(',')
)
alloc_exploded = (
    alloc_norm
    .explode('SiteUUID')
    .assign(SiteUUID=lambda d: d['SiteUUID'].str.strip())
)
alloc_exploded = alloc_exploded[alloc_exploded['SiteUUID'].notna() & (alloc_exploded['SiteUUID'] != '')]

# --- Merge with sites (keep only allocations that actually have a site)
wade_1 = pd.merge(alloc_exploded, sites_df, on='SiteUUID', how='inner')


if 'WaterSourceUUIDs' in wade_1.columns:
    wade_1['WaterSourceUUIDs'] = (
        wade_1['WaterSourceUUIDs']
        .astype(str)
        .str.split(',')
    )
    wade_1 = (
        wade_1
        .explode('WaterSourceUUIDs')
        .assign(WaterSourceUUID=lambda d: d['WaterSourceUUIDs'].str.strip())
        .drop(columns=['WaterSourceUUIDs'])
    )
elif 'WaterSourceUUID' in wade_1.columns:
    # Ensure clean strings if it's already singular
    wade_1['WaterSourceUUID'] = wade_1['WaterSourceUUID'].astype(str).str.strip()

if 'SiteNativeID' in wade_1.columns:
    wade_1 = wade_1.rename(columns={'SiteNativeID': 'WDID'})

wade_df = pd.merge(wade_1, source_df, on='WaterSourceUUID', how='left')

wade_df['WDID'] = wade_df['WDID'].astype(str)
diversions_df['WadeID'] = diversions_df['WadeID'].astype(str)
wade_df['WDID'] = wade_df['WDID'].str.strip()
diversions_df['WadeID'] = diversions_df['WadeID'].str.strip()
if os.path.exists(agg_csv):
    aggregated_diversions_df = pd.read_csv(agg_csv)
    aggregated_diversions_df['Aggregation ID'] = aggregated_diversions_df['Aggregation ID'].astype(str).str.strip()
else:
    print(" aggregated_table.csv not found. Continuing without aggregated diversion data.")
    # Make empty dataframe with expected columns so code doesn't break
    aggregated_diversions_df = pd.DataFrame(columns=[
        'Aggregation ID', 'Aggregation Name', 'Water Source'
    ])



In [ ]:
# Diversions Layer

diversions_df['geometry'] = [Point(xy) for xy in zip(diversions_df['X'], diversions_df['Y'])]
diversions_gdf = gpd.GeoDataFrame(diversions_df, geometry='geometry', crs='EPSG:4326')  # Assuming input is in lat/lon

diversions_gdf = diversions_gdf.to_crs(wbd_layer.crs)

wbd_selected = wbd_layer[wbd_layer[huc_feature] == huc_code]

diversions_selected = gpd.clip(diversions_gdf, wbd_selected)

diversions_selected = diversions_selected.drop(columns=['geometry', 'index_right'], errors='ignore').copy()
POD_df = diversions_selected.copy().drop(columns=['ID', 'State'], errors='')
POD_df.columns = ['WDID', 'LATITUDE', 'LONGITUDE','POI_NATIVE_ID']

POD_df['TYPE'] = None
POD_df['BENEFICIAL_CATEGORY_USE'] = 'TBD'
POD_df['SITE_NAME'] ='TBD'
POD_df['WATER_SOURCE'] ='TBD'
POD_df['SOURCE_GNIS_ID']='TBD'


wade_wdids = set(wade_df['WDID'].values)
agg_wdids  = set(aggregated_diversions_df['Aggregation ID'].values)

# Loop using itertuples
for row in tqdm(POD_df.itertuples(), total=len(POD_df), desc="Classifying PODs"):

    idx = row.Index
    diversion_id = row.WDID

    # Case 1 — Physical diversion
    if diversion_id in wade_wdids:
        match = wade_df.loc[wade_df['WDID'] == diversion_id].iloc[0]

        POD_df.at[idx, 'TYPE'] = 'Physical'
        POD_df.at[idx, 'SITE_NAME'] = match['SiteName']
        POD_df.at[idx, 'WATER_SOURCE'] = match['WaterSourceName']
        POD_df.at[idx, 'BENEFICIAL_CATEGORY_USE'] = match['BeneficialUseCategory']

    # Case 2 — Aggregated diversion
    elif diversion_id in agg_wdids:
        match = aggregated_diversions_df.loc[
            aggregated_diversions_df['Aggregation ID'] == diversion_id
        ].iloc[0]

        POD_df.at[idx, 'TYPE'] = 'Aggregated Diversion'
        POD_df.at[idx, 'SITE_NAME'] = match['Aggregation Name']
        POD_df.at[idx, 'WATER_SOURCE'] = match['Water Source']
        
POD_df = POD_df.dropna(subset=['TYPE'])  
POD_df.to_csv(f"data/output/PODtable_{huc_code}.csv", index=False)

In [ ]:
# --- POI Extraction Block ---

# Load POD table generated earlier in the notebook
POD_df = pd.read_csv(f'data/output/PODtable_{huc_code}.csv', dtype={'WDID': str})

# Filter to selected HUC2
huc_boundary = wbd_layer[wbd_layer[huc_feature] == huc_code]


# Merge reservoir attributes
resops_updated = resops_df.merge(
    grand_df.rename(columns={'GRAND_ID': 'DAM_ID'})[
        ['DAM_ID', 'RIVER', 'CAP_MCM', 'DAM_HGT_M', 'AREA_SKM', 'MAIN_USE']
    ],
    on='DAM_ID',
    how='left'
)

# Convert POD table to GDF
pod_gdf = gpd.GeoDataFrame(
    POD_df,
    geometry=gpd.points_from_xy(POD_df.LONGITUDE, POD_df.LATITUDE),
    crs="EPSG:4326"
)

# Convert reservoir CSV to GDF
resops_gdf = gpd.GeoDataFrame(
    resops_updated,
    geometry=gpd.points_from_xy(resops_updated.LONGITUDE, resops_updated.LATITUDE),
    crs="EPSG:4326"
)

# Reproject everything to match WBD layer
target_crs = huc_boundary.crs

pod_gdf    = pod_gdf.to_crs(target_crs)
resops_gdf = resops_gdf.to_crs(target_crs)
gage_shp   = gage_shp.to_crs(target_crs)
huc_boundary = huc_boundary.to_crs(target_crs)

# Select points inside the HUC boundary
pod_selected    = pod_gdf[pod_gdf.within(huc_boundary.unary_union)]
resops_selected = resops_gdf[resops_gdf.within(huc_boundary.unary_union)]
gage_selected   = gage_shp[gage_shp.within(huc_boundary.unary_union)]

# Output file
output_gpkg_path = f"data/output/POI_{huc_code}.gpkg"

# Write layers
pod_selected.to_file(output_gpkg_path, layer="DIVERSION_POINTS", driver="GPKG")
resops_selected.to_file(output_gpkg_path, layer="RESERVOIR_POINTS", driver="GPKG")
gage_selected.to_file(output_gpkg_path, layer="GAGE_POINTS", driver="GPKG")

print(f"Selected POI layers saved to: {output_gpkg_path}")


In [ ]:
POD_data = gpd.read_file(f'data/output/POI_{huc_code}.gpkg', layer='DIVERSION_POINTS')
res_data = gpd.read_file(f'data/output/POI_{huc_code}.gpkg', layer='RESERVOIR_POINTS')


target_crs = flow_layer.crs
POD_data = POD_data.to_crs(target_crs)
res_data = res_data.to_crs(target_crs)
flow_layer['gnis_id'] = pd.to_numeric(flow_layer['gnis_id'], errors='coerce')
flow_layer['streamorde'] = pd.to_numeric(flow_layer['streamorde'], errors='coerce')
POD_data['Source GNIS ID Raw'] = POD_data['SOURCE_GNIS_ID']

fallback3_count = 0
fallback3_ids = []
closest_features = []
updated_gnis_ids = []


for idx, point in tqdm(POD_data.iterrows(), total=len(POD_data), desc="Linking PODs IDs"):
    closest_segment = closest_segment_POD(point, flow_layer)
    closest_feature_value = closest_segment[feature_name] if closest_segment is not None else None
    POD_data.at[idx, 'WATER_SOURCE'] = closest_segment['gnis_name']
    POD_data.at[idx, 'SOURCE_GNIS_ID'] = closest_segment['gnis_id']
    closest_features.append(closest_feature_value)

debug_point = POD_data.iloc[11]
debug_geom = debug_point.geometry
debug_gnis_id = debug_point['SOURCE_GNIS_ID']
matching_segments = flow_layer[flow_layer['gnis_id'] == debug_gnis_id].copy()
debug_closest_segment = closest_segment_POD(debug_point, flow_layer)

POD_data['SOURCE_COMID'] = closest_features
#POD_data['POI_NativeID'] = POD_data['WDID']
POD_data = POD_data.drop(columns='Source GNIS ID Raw')
closest_features_res = []

for idx, point in tqdm(res_data.iterrows(),total=len(res_data), desc="Linking Reservoir IDs") :
    river_name = res_data.at[idx, 'RIVER']
    point_geom = point.geometry
    closest_segment = closest_segment_func2(point, flow_layer)
    closest_feature_value = closest_segment[feature_name] if closest_segment is not None else None
    closest_features_res.append(closest_feature_value)

res_data['SOURCE_COMID'] = closest_features_res
res_data['POI_NATIVE_ID'] = res_data['NID_ID']

#gage_data['SOURCE_COMID'] = np.nan
#gage_data.loc[gage_data['hl_reference'] == 'type_gages', 'Source_' + feature_name] = gage_data['hy_id']

#gage_data['POI_NativeID'] = np.nan
#gage_data.loc[gage_data['hl_reference'] == 'type_gages', 'POI_NativeID'] = gage_data['hl_link']

print(f"\nNumber of PODs that used fallback 3: {fallback3_count}")
print(f"\nNumber of Reservoirs that used fallback 3: {res_fallback_counter}")

enhanced_fabric_path = f'data/output/enhanced_reference_{huc_code}.gpkg'
shutil.copyfile(reference_fabric_path, enhanced_fabric_path)

# with fiona.Env():
#     fiona.remove(enhanced_fabric_path, layer='event')

#gage_data.to_file(enhanced_fabric_path, layer='event', driver='GPKG', mode='a')
POD_data.to_file(enhanced_fabric_path, layer='DIVERSION_POINTS', driver='GPKG', mode='a')
res_data.to_file(enhanced_fabric_path, layer='RESERVOIR_POINTS', driver='GPKG', mode='a')